# 0. Imports and Setting up Anthropic API Client

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [ ]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/NL/NL2.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Ple

In [ ]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [ ]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [ ]:
# Print response
print(response_p1)

Let's formulate the mathematical optimization model for this problem by defining the parameters, decision variables, objective function, and constraints.

Parameters:
- L: Maximum capacity of the mixture tank (80 l)

Decision Variables:
- x₁: Litres of the first component used
- x₂: Litres of the second component used

Objective Function:
Maximize the quality of the product, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres.

Maximize: ln(x₁ × x₂) - ∛(x₁ + 2x₂)

Constraints:
1. Stability constraint: The squared difference between the litres employed of each component and the mean litres utilised of both components should be below ten.
   (x₁ - (x₁ + x₂) / 2)² + (x₂ - (x₁ + x₂) / 2)² ≤ 10

2. Mixture tank capacity constraint: The sum of the litres of both components should not exceed the maximum capacity of t

# 2. Prompt 2: Write Code based on Mathematical Model

In [ ]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [ ]:
# Print response
print(response_p2)

Here's the Python code using the Pyomo library to solve the optimization problem:

```python
from pyomo.environ import *
import math

# Create a Pyomo model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2 * model.x2)**(1/3), sense=maximize)

# Define constraints
model.stability_constraint = Constraint(expr=(model.x1 - (model.x1 + model.x2) / 2)**2 + (model.x2 - (model.x1 + model.x2) / 2)**2 <= 10)
model.capacity_constraint = Constraint(expr=model.x1 + model.x2 <= 80)

# Solve the optimization problem
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", value(model.x1))
print("x2 =", value(model.x2))
print("Objective value =", value(model.obj))
```

Explanation:
1. We import the necessary modules from the Pyomo library and the `math` module.

2. We

# 4. Input Problem Data and Test Model Code

In [ ]:
# Installing pyomo nd solver
!pip install -q pyomo
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
from pyomo.environ import *
import math

# Create a Pyomo model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define objective function
model.obj = Objective(expr=log(model.x1 * model.x2) - (model.x1 + 2 * model.x2)**(1/3), sense=maximize)

# Define constraints
model.stability_constraint = Constraint(expr=(model.x1 - (model.x1 + model.x2) / 2)**2 + (model.x2 - (model.x1 + model.x2) / 2)**2 <= 10)
model.capacity_constraint = Constraint(expr=model.x1 + model.x2 <= 80)

# Solve the optimization problem
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", value(model.x1))
print("x2 =", value(model.x2))
print("Objective value =", value(model.obj))

Optimal solution:
x1 = 42.236067853543105
x2 = 37.76393234768411
Objective value = 2.473033919646447


# 5. Correct The Model Code to Test Mathematical Model (if applicable)